## Make regression data

In [ ]:
import gzip
import pickle
import sys
from pathlib import Path
from astropy.table import Table
import astropy.units as u
from tqdm import tqdm

sys.path.insert(0, str(Path.home() / "git" / "proseco"))
#sys.path.insert(0, str(Path.home() / "git" / "sparkles"))

import kadi.commands as kc
import matplotlib.pyplot as plt
import numpy as np
import parse_cm.paths
from astropy.table import Table
from cxotime import CxoTime

from proseco import get_aca_catalog
import sparkles


In [2]:
import proseco

print(proseco.__version__)

5.17.1


In [3]:
print(sparkles.__version__)

4.31.0


In [4]:
# Get the last 2.5 years of approved schedules
obss = kc.get_observations(CxoTime.now() - 365 * 2.5 * u.day)
obss = [obs for obs in obss if obs["source"] != "CMD_EVT"]
load_names = []
[load_names.append(obs["source"]) for obs in obss if obs["source"] not in load_names];


In [5]:
def print_acar(acar, load_name):
    lines = []
    lines.append("-" * 40)
    lines.append(f"ACA Review for {load_name}")
    lines.append(f"obsid: {acar.obsid}")
    lines.append(f"target_name: {acar.target_name}")
    lines.append(f"date: {acar.date}")
    lines.append(f"P2: {-np.log10(acar.acqs.calc_p_safe())}")
    lines.append(f"guide count: {acar.guide_count}")
    lines.extend(acar.pformat())
    for msg in acar.messages:
        lines.append(f"MSG: {msg}")
    text = "\n".join(lines)
    return text


In [ ]:
# make an output dir that includes proseco version and sparkles version
outdir = Path("regression_data") / f"{proseco.__version__}_{sparkles.__version__}"
outdir.mkdir(parents=True, exist_ok=True)

In [7]:

out_text_file = outdir / "catalogs.txt"
# Remove existing text file
if out_text_file.exists():
    out_text_file.unlink()
for load_name in tqdm(load_names):
    path_pkl_out = outdir / f"{load_name}_acas.pkl.gz"
    if path_pkl_out.exists():
        new_acas = pickle.load(gzip.open(path_pkl_out, "rb"))
        for obsid in new_acas:
            acar = new_acas[obsid].get_review_table()
            acar.run_aca_review()
            text = print_acar(acar, load_name)
            with open(out_text_file, "a") as f:
                f.write(text + "\n")
    else:
        path_pkl = parse_cm.paths.load_file_path(load_name, "output/*.pkl.gz")
        acas = pickle.load(gzip.open(path_pkl, "rb"))
        # Use a little table to keep everything in time order
        obsids = Table([{"date": acas[obsid].date, "obsid": obsid} for obsid in acas])
        obsids.sort("date")
        new_acas = {}
        for obsid in obsids:
            args = acas[obsid["obsid"]].call_args.copy()
            args["raise_exc"] = True
            has_edits = False
            for kwarg in ["include_ids_guide", "include_ids_acq", "include_halfws_acq",
                        "exclude_ids_guide", "exclude_ids_acq"]:
                if kwarg in args:
                    args.pop(kwarg)
                    has_edits = True
            if has_edits:
                # Do it once without the edits and again with the as-run edits
                new_aca = get_aca_catalog(**args)
                new_acas[obsid["obsid"] + .1] = new_aca
                acar = new_aca.get_review_table()
                acar.run_aca_review()
                text = print_acar(acar, load_name)
                with open(out_text_file, "a") as f:
                    f.write(text + "\n")
            # With edits it looks like there's more of a chance with a force-include
            # not working, so catch exceptions here
            try:
                new_aca = get_aca_catalog(**acas[obsid["obsid"]].call_args)
            except ValueError as e:
                text = f"ERROR for obsid {obsid['obsid']} load {load_name}: {e}"
                with open(out_text_file, "a") as f:
                    f.write(text + "\n")
                continue
            new_acas[obsid["obsid"]] = new_aca
            acar = new_aca.get_review_table()
            acar.run_aca_review()
            text = print_acar(acar, load_name)
            with open(out_text_file, "a") as f:
                f.write(text + "\n")

        with gzip.open(path_pkl_out, "wb") as f:
            pickle.dump(new_acas, f)


 91%|█████████ | 137/151 [21:01<01:55,  8.27s/it]/Users/jean/miniforge3/envs/ska3-latest/lib/python3.12/site-packages/chandra_aca/star_probs.py:438: UserWarning: 
Model grid-local-quadratic-2025-10.fits.gz computed between -15 <= t_ccd <= 2, clipping input t_ccd(s) outside that range.
  warnings.warn(
/Users/jean/miniforge3/envs/ska3-latest/lib/python3.12/site-packages/chandra_aca/star_probs.py:438: UserWarning: 
Model grid-local-quadratic-2025-10.fits.gz computed between -15 <= t_ccd <= 2, clipping input t_ccd(s) outside that range.
  warnings.warn(
/Users/jean/miniforge3/envs/ska3-latest/lib/python3.12/site-packages/chandra_aca/star_probs.py:438: UserWarning: 
Model grid-local-quadratic-2025-10.fits.gz computed between -15 <= t_ccd <= 2, clipping input t_ccd(s) outside that range.
  warnings.warn(
100%|██████████| 151/151 [23:06<00:00,  9.18s/it]
